In [3]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat
import pyomo.environ as pe

# Intervalo entorno a la media

Importar datos del problema

In [4]:
data = loadmat("Datos_Manufacturing_media.mat")
A_media = data['A_media']
A_hat_media = data['A_hat_media']
b = data['b'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 

In [5]:
# ─── Dimensiones ────────────────────────────────────────────────────────────────
M, N = A_media.shape  # M restricciones, N variables

# ─── Crear modelo ───────────────────────────────────────────────────────────────
m = pe.ConcreteModel()

# Variables de decisión
m.x = pe.Var(range(N), domain=pe.NonNegativeReals)
m.y = pe.Var(range(N), domain=pe.NonNegativeReals)  # y[j] ≥ |x[j]|

# ─── Función objetivo: maximizar cᵀx ─────────────────────────────────────────────
m.obj = pe.Objective(expr=sum(c[j] * m.x[j] for j in range(N)), sense=pe.maximize)

# ─── Relación y_j ≥ |x_j| ───────────────────────────────────────────────────────
for j in range(N):
    m.add_component(f"abs_x_pos_{j}", pe.Constraint(expr= m.y[j] >=  m.x[j]))
    m.add_component(f"abs_x_neg_{j}", pe.Constraint(expr= m.y[j] >= -m.x[j]))

# ─── Restricciones robustas (Soyster) ───────────────────────────────────────────
for i in range(M):
    nom = sum(A_media[i, j] * m.x[j] for j in range(N))
    dev = sum(A_hat_media[i, j] * m.y[j] for j in range(N))
    m.add_component(
        f"robust_con_{i}",
        pe.Constraint(expr= nom + dev <= b[i])
    )

# ─── Resolver el problema ───────────────────────────────────────────────────────
solver = pe.SolverFactory('gurobi')
result = solver.solve(m, tee=True)

# ─── Mostrar resultados ─────────────────────────────────────────────────────────
print("Termination:", result.solver.termination_condition)
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    for j in range(N):
        x_val = pe.value(m.x[j])
        print(f"x[{j}] = {x_val:.6e}")
    print(f"Obj = {pe.value(m.obj):.6e}")
else:
    print("No se encontró solución óptima.")

Read LP format model from file C:\Users\javie\AppData\Local\Temp\tmp9dm48pa9.pyomo.lp
Reading time = 0.00 seconds
x1: 13 rows, 10 columns, 50 nonzeros
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Optimize a model with 13 rows, 10 columns and 50 nonzeros
Model fingerprint: 0x3c685446
Coefficient statistics:
  Matrix range     [1e-01, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+02]
Presolve removed 11 rows and 5 columns
Presolve time: 0.00s
Presolved: 2 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.3822187e+00   0.000000e+00   0.000000e+00      0s
       0    6.3822187e+00   0.0000

# Intervalo [min max]

In [6]:
data = loadmat("Datos_Manufacturing_MinMax.mat")
A_MinMax = data['A_MinMax']
A_hat_MinMax = data['A_hat_MinMax']
b = data['b'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [7]:
# ─── Dimensiones ────────────────────────────────────────────────────────────────
M, N = A_MinMax.shape  # M restricciones, N variables

# ─── Crear modelo ───────────────────────────────────────────────────────────────
m = pe.ConcreteModel()

# Variables de decisión
m.x = pe.Var(range(N), domain=pe.NonNegativeReals)
m.y = pe.Var(range(N), domain=pe.NonNegativeReals)  # y[j] ≥ |x[j]|

# ─── Función objetivo: maximizar cᵀx ─────────────────────────────────────────────
m.obj = pe.Objective(expr=sum(c[j] * m.x[j] for j in range(N)), sense=pe.maximize)

# ─── Relación y_j ≥ |x_j| ───────────────────────────────────────────────────────
for j in range(N):
    m.add_component(f"abs_x_pos_{j}", pe.Constraint(expr= m.y[j] >=  m.x[j]))
    m.add_component(f"abs_x_neg_{j}", pe.Constraint(expr= m.y[j] >= -m.x[j]))

# ─── Restricciones robustas (Soyster) ───────────────────────────────────────────
for i in range(M):
    nom = sum(A_MinMax[i, j] * m.x[j] for j in range(N))
    dev = sum(A_hat_MinMax[i, j] * m.y[j] for j in range(N))
    m.add_component(
        f"robust_con_{i}",
        pe.Constraint(expr= nom + dev <= b[i])
    )

# ─── Resolver el problema ───────────────────────────────────────────────────────
solver = pe.SolverFactory('gurobi')
result = solver.solve(m, tee=True)

# ─── Mostrar resultados ─────────────────────────────────────────────────────────
print("Termination:", result.solver.termination_condition)
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    for j in range(N):
        x_val = pe.value(m.x[j])
        print(f"x[{j}] = {x_val:.6e}")
    print(f"Obj = {pe.value(m.obj):.6e}")
else:
    print("No se encontró solución óptima.")

Read LP format model from file C:\Users\javie\AppData\Local\Temp\tmp7g7o603f.pyomo.lp
Reading time = 0.01 seconds
x1: 13 rows, 10 columns, 50 nonzeros
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Optimize a model with 13 rows, 10 columns and 50 nonzeros


Model fingerprint: 0x9633eac8
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+02]
Presolve removed 11 rows and 5 columns
Presolve time: 0.00s
Presolved: 2 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.000000000e+00
Termination: optimal
x[0] = 4.000000e+00
x[1] = 0.000000e+00
x[2] = 0.000000e+00
x[3] = 0.000000e+00
x[4] = 0.000000e+00
Obj = 4.000000e+00
